## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

## Demo numpy based auto differentiation

In [15]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        x = self.mem['x']
        W = self.mem['W']
        
        grad_x = np.dot(grad_y, W.T)
        grad_W = np.dot(x.T, grad_y)
        
        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        grad_y = np.where(self.mem['x'] > 0, grad_y, np.zeros_like(grad_y))
        grad_x = grad_y
        return grad_x
    


class Softmax:
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        out = np.log(x+self.epsilon)
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


# Final Gradient Check

In [12]:
import tensorflow as tf

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(h2_log_grad)
print('--'*20)
# print(W2_grad)

with tf.GradientTape() as tape:
    x, W1, W2, label = tf.constant(x), tf.constant(W1), tf.constant(W2), tf.constant(label)
    tape.watch(W1)
    tape.watch(W2)
    h1 = tf.matmul(x, W1)
    h1_relu = tf.nn.relu(h1)
    h2 = tf.matmul(h1_relu, W2)
    prob = tf.nn.softmax(h2)
    log_prob = tf.math.log(prob)
    loss = tf.reduce_sum(label * log_prob)
    grads = tape.gradient(loss, [prob])
    print (grads[0].numpy())

[[ 0.          6.01226766  0.          0.          0.          0.        ]
 [11.99272591  0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          3.15438893  0.          0.        ]
 [ 0.          0.          0.          0.          0.         14.09081649]
 [53.67410584  0.          0.          0.          0.          0.        ]]
----------------------------------------
[[ 0.          6.01226766  0.          0.          0.          0.        ]
 [11.99272591  0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          3.15438893  0.          0.        ]
 [ 0.          0.          0.          0.          0.         14.09081649]
 [53.67410584  0.          0.          0.          0.          0.        ]]


## 建立模型

In [13]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss

In [14]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [18]:
train_data, test_data = mnist_dataset()
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(50):
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

epoch 0 : loss 4.134336432782026 ; accuracy 0.8035166666666667
epoch 1 : loss 4.0407316803185385 ; accuracy 0.8045833333333333
epoch 2 : loss 3.90873071402988 ; accuracy 0.81265
epoch 3 : loss 3.8369528163615536 ; accuracy 0.81345
epoch 4 : loss 3.7436517614922864 ; accuracy 0.8194
epoch 5 : loss 3.7017832746770933 ; accuracy 0.8191666666666667
epoch 6 : loss 3.6193973579677987 ; accuracy 0.8240166666666666
epoch 7 : loss 3.5928057976646772 ; accuracy 0.8237333333333333
epoch 8 : loss 3.5214215442341823 ; accuracy 0.8273
epoch 9 : loss 3.5009909748546537 ; accuracy 0.82755
epoch 10 : loss 3.433789335604436 ; accuracy 0.831
epoch 11 : loss 3.4172811542661017 ; accuracy 0.8308166666666666
epoch 12 : loss 3.358621086534705 ; accuracy 0.83405
epoch 13 : loss 3.335163016563896 ; accuracy 0.8337
epoch 14 : loss 3.2877895134502 ; accuracy 0.8368
epoch 15 : loss 3.2577226298104844 ; accuracy 0.8369
epoch 16 : loss 3.2250418637025624 ; accuracy 0.8399166666666666
epoch 17 : loss 3.1918256364572

In [20]:
((217.6-34.2)*1500+1400*1880)*0.9/(51000*0.71)

72.25600662800332